In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
#import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


# Max, Min values for range normalization

# Focal Length (fx)

## Generated Data

In [7]:
afx = np.load("./Data/fx.npy")
afx = afx[int(len(afx)*0.7):]
pfx_avg = np.load("./Data/p_avg_gd.npy")[0]
pfx_dh = np.load("./Data/p_dh_gd.npy")
pfx_cplb = np.load("./Data/p_cplb_gd.npy")
pfx_cplu = np.load("./Data/p_cplu_gd.npy")
pfx_cpla = np.load("./Data/p_cpla_gd.npy")

In [20]:
len(afx), len(pfx_cpla[0]), len(pfx_dh[0])

(23796, 23796, 13)

In [31]:
max_fx_l = 0
min_fx_l = 100000

for idx in range(len(afx)):
    
    error_avg = abs(pfx_avg - afx[idx])
    error_dh = abs(pfx_dh[idx][0] - afx[idx])
    error_cplb = abs(pfx_cplb[0][idx] - afx[idx])
    error_cplu = abs(pfx_cplu[0][idx] - afx[idx])
    error_cpla = abs(pfx_cpla[0][idx] - afx[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_fx_l:
        
        min_fx_l = t_min
        
    if t_max > max_fx_l:
        
        max_fx_l = t_max
    

In [32]:
min_fx_l, max_fx_l

(array([1.9073486e-06], dtype=float32), array([3214.9907], dtype=float32))

# Real Data

In [35]:
afx = np.load("./Data/fxr.npy")
pfx_avg = np.load("./Data/p_avg_ad.npy")[0]
pfx_dh = np.load("./Data/p_dh_ad.npy")
pfx_cplb = np.load("./Data/p_cplb_ad.npy")
pfx_cplu = np.load("./Data/p_cplu_ad.npy")
pfx_cpla = np.load("./Data/p_cpla_ad.npy")

In [36]:
len(afx), len(pfx_cpla[0]), len(pfx_dh[0])

(2914, 2914, 13)

In [37]:
max_fx_l = 0
min_fx_l = 100000

for idx in range(len(afx)):
    
    error_avg = abs(pfx_avg - afx[idx])
    error_dh = abs(pfx_dh[idx][0] - afx[idx])
    error_cplb = abs(pfx_cplb[0][idx] - afx[idx])
    error_cplu = abs(pfx_cplu[0][idx] - afx[idx])
    error_cpla = abs(pfx_cpla[0][idx] - afx[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_fx_l:
        
        min_fx_l = t_min
        
    if t_max > max_fx_l:
        
        max_fx_l = t_max
    

In [38]:
min_fx_l, max_fx_l

(array([1373.2275], dtype=float32), 2269.381661995158)

# Focal Length (fy)

In [42]:
afy = np.load("./Data/fy.npy")
afy = afy[int(len(afy)*0.7):]
pfy_avg = np.load("./Data/p_avg_gd.npy")[1]
pfy_dh = np.load("./Data/p_dh_gd.npy")
pfy_cplb = np.load("./Data/p_cplb_gd.npy")
pfy_cplu = np.load("./Data/p_cplu_gd.npy")
pfy_cpla = np.load("./Data/p_cpla_gd.npy")

max_fy_l = 0
min_fy_l = 100000

for idx in range(len(afy)):
    
    error_avg = abs(pfy_avg - afy[idx])
    error_dh = abs(pfy_dh[idx][1] - afy[idx])
    error_cplb = abs(pfy_cplb[1][idx] - afy[idx])
    error_cplu = abs(pfy_cplu[1][idx] - afy[idx])
    error_cpla = abs(pfy_cpla[1][idx] - afy[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_fy_l:
        
        min_fy_l = t_min
        
    if t_max > max_fy_l:
        
        max_fy_l = t_max
    
print ("gd min loss: ", min_fy_l, "gd max loss: ", max_fy_l)

afy = np.load("./Data/fyr.npy")
pfy_avg = np.load("./Data/p_avg_ad.npy")[1]
pfy_dh = np.load("./Data/p_dh_ad.npy")
pfy_cplb = np.load("./Data/p_cplb_ad.npy")
pfy_cplu = np.load("./Data/p_cplu_ad.npy")
pfy_cpla = np.load("./Data/p_cpla_ad.npy")

max_fy_l = 0
min_fy_l = 100000

for idx in range(len(afy)):
    
    error_avg = abs(pfy_avg - afy[idx])
    error_dh = abs(pfy_dh[idx][1] - afy[idx])
    error_cplb = abs(pfy_cplb[1][idx] - afy[idx])
    error_cplu = abs(pfy_cplu[1][idx] - afy[idx])
    error_cpla = abs(pfy_cpla[1][idx] - afy[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_fy_l:
        
        min_fy_l = t_min
        
    if t_max > max_fy_l:
        
        max_fy_l = t_max
    
print ("ad min loss: ", min_fy_l, "ad max loss: ", max_fy_l)

gd min loss:  [2.9563904e-05] gd max loss:  [2051.209]
ad min loss:  [1520.9558] ad max loss:  [2268.491]


# u0

In [43]:
au0 = np.load("./Data/u0.npy")
au0 = au0[int(len(au0)*0.7):]
pu0_avg = np.load("./Data/p_avg_gd.npy")[2]
pu0_dh = np.load("./Data/p_dh_gd.npy")
pu0_cplb = np.load("./Data/p_cplb_gd.npy")
pu0_cplu = np.load("./Data/p_cplu_gd.npy")
pu0_cpla = np.load("./Data/p_cpla_gd.npy")

max_u0_l = 0
min_u0_l = 100000

for idx in range(len(au0)):
    
    error_avg = abs(pu0_avg - au0[idx])
    error_dh = abs(pu0_dh[idx][2] - au0[idx])
    error_cplb = abs(pu0_cplb[2][idx] - au0[idx])
    error_cplu = abs(pu0_cplu[2][idx] - au0[idx])
    error_cpla = abs(pu0_cpla[2][idx] - au0[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_u0_l:
        
        min_u0_l = t_min
        
    if t_max > max_u0_l:
        
        max_u0_l = t_max
    
print ("gd min loss: ", min_u0_l, "gd max loss: ", max_u0_l)

au0 = np.load("./Data/u0r.npy")
pu0_avg = np.load("./Data/p_avg_ad.npy")[2]
pu0_dh = np.load("./Data/p_dh_ad.npy")
pu0_cplb = np.load("./Data/p_cplb_ad.npy")
pu0_cplu = np.load("./Data/p_cplu_ad.npy")
pu0_cpla = np.load("./Data/p_cpla_ad.npy")

max_u0_l = 0
min_u0_l = 100000

for idx in range(len(au0)):
    
    error_avg = abs(pu0_avg - au0[idx])
    error_dh = abs(pu0_dh[idx][2] - au0[idx])
    error_cplb = abs(pu0_cplb[2][idx] - au0[idx])
    error_cplu = abs(pu0_cplu[2][idx] - au0[idx])
    error_cpla = abs(pu0_cpla[2][idx] - au0[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_u0_l:
        
        min_u0_l = t_min
        
    if t_max > max_u0_l:
        
        max_u0_l = t_max
    
print ("ad min loss: ", min_u0_l, "ad max loss: ", max_u0_l)

gd min loss:  [1.9073486e-05] gd max loss:  [2178.9983]
ad min loss:  [382.5896] ad max loss:  1010.2414896422663


# v0

In [45]:
av0 = np.load("./Data/v0.npy")
av0 = av0[int(len(av0)*0.7):]
pv0_avg = np.load("./Data/p_avg_gd.npy")[3]
pv0_dh = np.load("./Data/p_dh_gd.npy")
pv0_cplb = np.load("./Data/p_cplb_gd.npy")
pv0_cplu = np.load("./Data/p_cplu_gd.npy")
pv0_cpla = np.load("./Data/p_cpla_gd.npy")

max_v0_l = 0
min_v0_l = 100000

for idx in range(len(av0)):
    
    error_avg = abs(pv0_avg - av0[idx])
    error_dh = abs(pv0_dh[idx][3] - av0[idx])
    error_cplb = abs(pv0_cplb[3][idx] - av0[idx])
    error_cplu = abs(pv0_cplu[3][idx] - av0[idx])
    error_cpla = abs(pv0_cpla[3][idx] - av0[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_v0_l:
        
        min_v0_l = t_min
        
    if t_max > max_v0_l:
        
        max_v0_l = t_max
    
print ("gd min loss: ", min_v0_l, "gd max loss: ", max_v0_l)

av0 = np.load("./Data/v0r.npy")
pv0_avg = np.load("./Data/p_avg_ad.npy")[3]
pv0_dh = np.load("./Data/p_dh_ad.npy")
pv0_cplb = np.load("./Data/p_cplb_ad.npy")
pv0_cplu = np.load("./Data/p_cplu_ad.npy")
pv0_cpla = np.load("./Data/p_cpla_ad.npy")

max_v0_l = 0
min_v0_l = 100000

for idx in range(len(av0)):
    
    error_avg = abs(pv0_avg - av0[idx])
    error_dh = abs(pv0_dh[idx][3] - av0[idx])
    error_cplb = abs(pv0_cplb[3][idx] - av0[idx])
    error_cplu = abs(pv0_cplu[3][idx] - av0[idx])
    error_cpla = abs(pv0_cpla[3][idx] - av0[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_v0_l:
        
        min_v0_l = t_min
        
    if t_max > max_v0_l:
        
        max_v0_l = t_max
    
print ("ad min loss: ", min_v0_l, "ad max loss: ", max_v0_l)

gd min loss:  [3.8146973e-06] gd max loss:  [2385.7239]
ad min loss:  [33.014587] ad max loss:  504.2467619374397


# baseline

In [46]:
ab = np.load("./Data/b.npy")
ab = ab[int(len(ab)*0.7):]
pb_avg = np.load("./Data/p_avg_gd.npy")[4]
pb_dh = np.load("./Data/p_dh_gd.npy")
pb_cplb = np.load("./Data/p_cplb_gd.npy")
pb_cplu = np.load("./Data/p_cplu_gd.npy")
pb_cpla = np.load("./Data/p_cpla_gd.npy")

max_b_l = 0
min_b_l = 100000

for idx in range(len(ab)):
    
    error_avg = abs(pb_avg - ab[idx])
    error_dh = abs(pb_dh[idx][4] - ab[idx])
    error_cplb = abs(pb_cplb[4][idx] - ab[idx])
    error_cplu = abs(pb_cplu[4][idx] - ab[idx])
    error_cpla = abs(pb_cpla[4][idx] - ab[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_b_l:
        
        min_b_l = t_min
        
    if t_max > max_b_l:
        
        max_b_l = t_max
    
print ("gd min loss: ", min_b_l, "gd max loss: ", max_b_l)

ab = np.load("./Data/br.npy")
pb_avg = np.load("./Data/p_avg_ad.npy")[4]
pb_dh = np.load("./Data/p_dh_ad.npy")
pb_cplb = np.load("./Data/p_cplb_ad.npy")
pb_cplu = np.load("./Data/p_cplu_ad.npy")
pb_cpla = np.load("./Data/p_cpla_ad.npy")

max_b_l = 0
min_b_l = 100000

for idx in range(len(ab)):
    
    error_avg = abs(pb_avg - ab[idx])
    error_dh = abs(pb_dh[idx][4] - ab[idx])
    error_cplb = abs(pb_cplb[4][idx] - ab[idx])
    error_cplu = abs(pb_cplu[4][idx] - ab[idx])
    error_cpla = abs(pb_cpla[4][idx] - ab[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_b_l:
        
        min_b_l = t_min
        
    if t_max > max_b_l:
        
        max_b_l = t_max
    
print ("ad min loss: ", min_b_l, "ad max loss: ", max_b_l)

gd min loss:  [4.7683716e-06] gd max loss:  22.68721567997021
ad min loss:  [0.01699097] ad max loss:  23.478404679970208


# disparity


In [48]:
ad = np.load("./Data/d.npy")
ad = ad[int(len(ad)*0.7):]
pd_avg = np.load("./Data/p_avg_gd.npy")[5]
pd_dh = np.load("./Data/p_dh_gd.npy")
pd_cplb = np.load("./Data/p_cplb_gd.npy")
pd_cplu = np.load("./Data/p_cplu_gd.npy")
pd_cpla = np.load("./Data/p_cpla_gd.npy")

max_d_l = 0
min_d_l = 100000

for idx in range(len(ad)):
    
    error_avg = abs(pd_avg - ad[idx])
    error_dh = abs(pd_dh[idx][5] - ad[idx])
    error_cplb = abs(pd_cplb[5][idx] - ad[idx])
    error_cplu = abs(pd_cplu[5][idx] - ad[idx])
    error_cpla = abs(pd_cpla[5][idx] - ad[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_d_l:
        
        min_d_l = t_min
        
    if t_max > max_d_l:
        
        max_d_l = t_max
    
print ("gd min loss: ", min_d_l, "gd max loss: ", max_d_l)

ad = np.load("./Data/dr.npy")
pd_avg = np.load("./Data/p_avg_ad.npy")[5]
pd_dh = np.load("./Data/p_dh_ad.npy")
pd_cplb = np.load("./Data/p_cplb_ad.npy")
pd_cplu = np.load("./Data/p_cplu_ad.npy")
pd_cpla = np.load("./Data/p_cpla_ad.npy")

max_d_l = 0
min_d_l = 100000

for idx in range(len(ad)):
    
    error_avg = abs(pd_avg - ad[idx])
    error_dh = abs(pd_dh[idx][5] - ad[idx])
    error_cplb = abs(pd_cplb[5][idx] - ad[idx])
    error_cplu = abs(pd_cplu[5][idx] - ad[idx])
    error_cpla = abs(pd_cpla[5][idx] - ad[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_d_l:
        
        min_d_l = t_min
        
    if t_max > max_d_l:
        
        max_d_l = t_max
    
print ("ad min loss: ", min_d_l, "ad max loss: ", max_d_l)

gd min loss:  [9.536743e-07] gd max loss:  [134.60031]
ad min loss:  0.00021930835792538517 ad max loss:  [88.691475]


# tx


In [50]:
atx = np.load("./Data/tx.npy")
atx = atx[int(len(atx)*0.7):]
ptx_avg = np.load("./Data/p_avg_gd.npy")[6]
ptx_dh = np.load("./Data/p_dh_gd.npy")
ptx_cplb = np.load("./Data/p_cplb_gd.npy")
ptx_cplu = np.load("./Data/p_cplu_gd.npy")
ptx_cpla = np.load("./Data/p_cpla_gd.npy")

max_tx_l = 0
min_tx_l = 100000

for idx in range(len(atx)):
    
    error_avg = abs(ptx_avg - atx[idx])
    error_dh = abs(ptx_dh[idx][6] - atx[idx])
    error_cplb = abs(ptx_cplb[6][idx] - atx[idx])
    error_cplu = abs(ptx_cplu[6][idx] - atx[idx])
    error_cpla = abs(ptx_cpla[6][idx] - atx[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_tx_l:
        
        min_tx_l = t_min
        
    if t_max > max_tx_l:
        
        max_tx_l = t_max
    
print ("gd min loss: ", min_tx_l, "gd max loss: ", max_tx_l)

atx = np.load("./Data/txr.npy")
ptx_avg = np.load("./Data/p_avg_ad.npy")[6]
ptx_dh = np.load("./Data/p_dh_ad.npy")
ptx_cplb = np.load("./Data/p_cplb_ad.npy")
ptx_cplu = np.load("./Data/p_cplu_ad.npy")
ptx_cpla = np.load("./Data/p_cpla_ad.npy")

max_tx_l = 0
min_tx_l = 100000

for idx in range(len(atx)):
    
    error_avg = abs(ptx_avg - atx[idx])
    error_dh = abs(ptx_dh[idx][6] - atx[idx])
    error_cplb = abs(ptx_cplb[6][idx] - atx[idx])
    error_cplu = abs(ptx_cplu[6][idx] - atx[idx])
    error_cpla = abs(ptx_cpla[6][idx] - atx[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_tx_l:
        
        min_tx_l = t_min
        
    if t_max > max_tx_l:
        
        max_tx_l = t_max
    
print ("ad min loss: ", min_tx_l, "ad max loss: ", max_tx_l)

gd min loss:  2.384185791015625e-06 gd max loss:  22.68721567997021
ad min loss:  2.6702880859375e-05 ad max loss:  21.68721567997021


# ty


In [51]:
aty = np.load("./Data/ty.npy")
aty = aty[int(len(aty)*0.7):]
pty_avg = np.load("./Data/p_avg_gd.npy")[7]
pty_dh = np.load("./Data/p_dh_gd.npy")
pty_cplb = np.load("./Data/p_cplb_gd.npy")
pty_cplu = np.load("./Data/p_cplu_gd.npy")
pty_cpla = np.load("./Data/p_cpla_gd.npy")

max_ty_l = 0
min_ty_l = 100000

for idx in range(len(aty)):
    
    error_avg = abs(pty_avg - aty[idx])
    error_dh = abs(pty_dh[idx][7] - aty[idx])
    error_cplb = abs(pty_cplb[7][idx] - aty[idx])
    error_cplu = abs(pty_cplu[7][idx] - aty[idx])
    error_cpla = abs(pty_cpla[7][idx] - aty[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_ty_l:
        
        min_ty_l = t_min
        
    if t_max > max_ty_l:
        
        max_ty_l = t_max
    
print ("gd min loss: ", min_ty_l, "gd max loss: ", max_ty_l)

aty = np.load("./Data/tyr.npy")
pty_avg = np.load("./Data/p_avg_ad.npy")[7]
pty_dh = np.load("./Data/p_dh_ad.npy")
pty_cplb = np.load("./Data/p_cplb_ad.npy")
pty_cplu = np.load("./Data/p_cplu_ad.npy")
pty_cpla = np.load("./Data/p_cpla_ad.npy")

max_ty_l = 0
min_ty_l = 100000

for idx in range(len(aty)):
    
    error_avg = abs(pty_avg - aty[idx])
    error_dh = abs(pty_dh[idx][7] - aty[idx])
    error_cplb = abs(pty_cplb[7][idx] - aty[idx])
    error_cplu = abs(pty_cplu[7][idx] - aty[idx])
    error_cpla = abs(pty_cpla[7][idx] - aty[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_ty_l:
        
        min_ty_l = t_min
        
    if t_max > max_ty_l:
        
        max_ty_l = t_max
    
print ("ad min loss: ", min_ty_l, "ad max loss: ", max_ty_l)

gd min loss:  [2.3841858e-07] gd max loss:  33.11693576309983
ad min loss:  [0.8303014] ad max loss:  33.99193576309983


# tz

In [52]:
atz = np.load("./Data/tz.npy")
atz = atz[int(len(atz)*0.7):]
ptz_avg = np.load("./Data/p_avg_gd.npy")[8]
ptz_dh = np.load("./Data/p_dh_gd.npy")
ptz_cplb = np.load("./Data/p_cplb_gd.npy")
ptz_cplu = np.load("./Data/p_cplu_gd.npy")
ptz_cpla = np.load("./Data/p_cpla_gd.npy")

max_tz_l = 0
min_tz_l = 100000

for idx in range(len(atz)):
    
    error_avg = abs(ptz_avg - atz[idx])
    error_dh = abs(ptz_dh[idx][8] - atz[idx])
    error_cplb = abs(ptz_cplb[8][idx] - atz[idx])
    error_cplu = abs(ptz_cplu[8][idx] - atz[idx])
    error_cpla = abs(ptz_cpla[8][idx] - atz[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_tz_l:
        
        min_tz_l = t_min
        
    if t_max > max_tz_l:
        
        max_tz_l = t_max
    
print ("gd min loss: ", min_tz_l, "gd max loss: ", max_tz_l)

atz = np.load("./Data/tzr.npy")
ptz_avg = np.load("./Data/p_avg_ad.npy")[8]
ptz_dh = np.load("./Data/p_dh_ad.npy")
ptz_cplb = np.load("./Data/p_cplb_ad.npy")
ptz_cplu = np.load("./Data/p_cplu_ad.npy")
ptz_cpla = np.load("./Data/p_cpla_ad.npy")

max_tz_l = 0
min_tz_l = 100000

for idx in range(len(atz)):
    
    error_avg = abs(ptz_avg - atz[idx])
    error_dh = abs(ptz_dh[idx][8] - atz[idx])
    error_cplb = abs(ptz_cplb[8][idx] - atz[idx])
    error_cplu = abs(ptz_cplu[8][idx] - atz[idx])
    error_cpla = abs(ptz_cpla[8][idx] - atz[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_tz_l:
        
        min_tz_l = t_min
        
    if t_max > max_tz_l:
        
        max_tz_l = t_max
    
print ("ad min loss: ", min_tz_l, "ad max loss: ", max_tz_l)

gd min loss:  [2.3841858e-06] gd max loss:  17.20185265614626
ad min loss:  0.0011047315597534357 ad max loss:  16.97185265614626


# pitch

In [53]:
ap = np.load("./Data/p.npy")
ap = ap[int(len(ap)*0.7):]
pp_avg = np.load("./Data/p_avg_gd.npy")[9]
pp_dh = np.load("./Data/p_dh_gd.npy")
pp_cplb = np.load("./Data/p_cplb_gd.npy")
pp_cplu = np.load("./Data/p_cplu_gd.npy")
pp_cpla = np.load("./Data/p_cpla_gd.npy")

max_p_l = 0
min_p_l = 100000

for idx in range(len(ap)):
    
    error_avg = abs(pp_avg - ap[idx])
    error_dh = abs(pp_dh[idx][9] - ap[idx])
    error_cplb = abs(pp_cplb[9][idx] - ap[idx])
    error_cplu = abs(pp_cplu[9][idx] - ap[idx])
    error_cpla = abs(pp_cpla[9][idx] - ap[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_p_l:
        
        min_p_l = t_min
        
    if t_max > max_p_l:
        
        max_p_l = t_max
    
print ("gd min loss: ", min_p_l, "gd max loss: ", max_p_l)

ap = np.load("./Data/pr.npy")
pp_avg = np.load("./Data/p_avg_ad.npy")[9]
pp_dh = np.load("./Data/p_dh_ad.npy")
pp_cplb = np.load("./Data/p_cplb_ad.npy")
pp_cplu = np.load("./Data/p_cplu_ad.npy")
pp_cpla = np.load("./Data/p_cpla_ad.npy")

max_p_l = 0
min_p_l = 100000

for idx in range(len(ap)):
    
    error_avg = abs(pp_avg - ap[idx])
    error_dh = abs(pp_dh[idx][9] - ap[idx])
    error_cplb = abs(pp_cplb[9][idx] - ap[idx])
    error_cplu = abs(pp_cplu[9][idx] - ap[idx])
    error_cpla = abs(pp_cpla[9][idx] - ap[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_p_l:
        
        min_p_l = t_min
        
    if t_max > max_p_l:
        
        max_p_l = t_max
    
print ("ad min loss: ", min_p_l, "ad max loss: ", max_p_l)

gd min loss:  4.57763671875e-05 gd max loss:  [4502.9224]
ad min loss:  [10.154588] ad max loss:  [790.04095]


# xw


In [54]:
axw = np.load("./Data/x.npy")
axw = axw[int(len(axw)*0.7):]
pxw_avg = np.load("./Data/p_avg_gd.npy")[10]
pxw_dh = np.load("./Data/p_dh_gd.npy")
pxw_cplb = np.load("./Data/p_cplb_gd.npy")
pxw_cplu = np.load("./Data/p_cplu_gd.npy")
pxw_cpla = np.load("./Data/p_cpla_gd.npy")

max_xw_l = 0
min_xw_l = 100000

for idx in range(len(axw)):
    
    error_avg = abs(pxw_avg - axw[idx])
    error_dh = abs(pxw_dh[idx][10] - axw[idx])
    error_cplb = abs(pxw_cplb[10][idx] - axw[idx])
    error_cplu = abs(pxw_cplu[10][idx] - axw[idx])
    error_cpla = abs(pxw_cpla[10][idx] - axw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_xw_l:
        
        min_xw_l = t_min
        
    if t_max > max_xw_l:
        
        max_xw_l = t_max
    
print ("gd min loss: ", min_xw_l, "gd max loss: ", max_xw_l)

axw = np.load("./Data/xr.npy")
pxw_avg = np.load("./Data/p_avg_ad.npy")[10]
pxw_dh = np.load("./Data/p_dh_ad.npy")
pxw_cplb = np.load("./Data/p_cplb_ad.npy")
pxw_cplu = np.load("./Data/p_cplu_ad.npy")
pxw_cpla = np.load("./Data/p_cpla_ad.npy")

max_xw_l = 0
min_xw_l = 100000

for idx in range(len(axw)):
    
    error_avg = abs(pxw_avg - axw[idx])
    error_dh = abs(pxw_dh[idx][10] - axw[idx])
    error_cplb = abs(pxw_cplb[10][idx] - axw[idx])
    error_cplu = abs(pxw_cplu[10][idx] - axw[idx])
    error_cpla = abs(pxw_cpla[10][idx] - axw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_xw_l:
        
        min_xw_l = t_min
        
    if t_max > max_xw_l:
        
        max_xw_l = t_max
    
print ("ad min loss: ", min_xw_l, "ad max loss: ", max_xw_l)

gd min loss:  5.219264654243716e-06 gd max loss:  [386.4486]
ad min loss:  [0.00066566] ad max loss:  [6457.3228]


# yw

In [55]:
ayw = np.load("./Data/y.npy")
ayw = ayw[int(len(ayw)*0.7):]
pyw_avg = np.load("./Data/p_avg_gd.npy")[11]
pyw_dh = np.load("./Data/p_dh_gd.npy")
pyw_cplb = np.load("./Data/p_cplb_gd.npy")
pyw_cplu = np.load("./Data/p_cplu_gd.npy")
pyw_cpla = np.load("./Data/p_cpla_gd.npy")

max_yw_l = 0
min_yw_l = 100000

for idx in range(len(ayw)):
    
    error_avg = abs(pyw_avg - ayw[idx])
    error_dh = abs(pyw_dh[idx][11] - ayw[idx])
    error_cplb = abs(pyw_cplb[11][idx] - ayw[idx])
    error_cplu = abs(pyw_cplu[11][idx] - ayw[idx])
    error_cpla = abs(pyw_cpla[11][idx] - ayw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_yw_l:
        
        min_yw_l = t_min
        
    if t_max > max_yw_l:
        
        max_yw_l = t_max
    
print ("gd min loss: ", min_yw_l, "gd max loss: ", max_yw_l)

ayw = np.load("./Data/yr.npy")
pyw_avg = np.load("./Data/p_avg_ad.npy")[11]
pyw_dh = np.load("./Data/p_dh_ad.npy")
pyw_cplb = np.load("./Data/p_cplb_ad.npy")
pyw_cplu = np.load("./Data/p_cplu_ad.npy")
pyw_cpla = np.load("./Data/p_cpla_ad.npy")

max_yw_l = 0
min_yw_l = 100000

for idx in range(len(ayw)):
    
    error_avg = abs(pyw_avg - ayw[idx])
    error_dh = abs(pyw_dh[idx][11] - ayw[idx])
    error_cplb = abs(pyw_cplb[11][idx] - ayw[idx])
    error_cplu = abs(pyw_cplu[11][idx] - ayw[idx])
    error_cpla = abs(pyw_cpla[11][idx] - ayw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_yw_l:
        
        min_yw_l = t_min
        
    if t_max > max_yw_l:
        
        max_yw_l = t_max
    
print ("ad min loss: ", min_yw_l, "ad max loss: ", max_yw_l)

gd min loss:  [3.8146973e-06] gd max loss:  [1835.1849]
ad min loss:  0.0036399107805458186 ad max loss:  [2944.4214]


# zw

In [56]:
azw = np.load("./Data/z.npy")
azw = azw[int(len(azw)*0.7):]
pzw_avg = np.load("./Data/p_avg_gd.npy")[12]
pzw_dh = np.load("./Data/p_dh_gd.npy")
pzw_cplb = np.load("./Data/p_cplb_gd.npy")
pzw_cplu = np.load("./Data/p_cplu_gd.npy")
pzw_cpla = np.load("./Data/p_cpla_gd.npy")

max_zw_l = 0
min_zw_l = 100000

for idx in range(len(azw)):
    
    error_avg = abs(pzw_avg - azw[idx])
    error_dh = abs(pzw_dh[idx][12] - azw[idx])
    error_cplb = abs(pzw_cplb[12][idx] - azw[idx])
    error_cplu = abs(pzw_cplu[12][idx] - azw[idx])
    error_cpla = abs(pzw_cpla[12][idx] - azw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_zw_l:
        
        min_zw_l = t_min
        
    if t_max > max_zw_l:
        
        max_zw_l = t_max
    
print ("gd min loss: ", min_zw_l, "gd max loss: ", max_zw_l)

azw = np.load("./Data/zr.npy")
pzw_avg = np.load("./Data/p_avg_ad.npy")[12]
pzw_dh = np.load("./Data/p_dh_ad.npy")
pzw_cplb = np.load("./Data/p_cplb_ad.npy")
pzw_cplu = np.load("./Data/p_cplu_ad.npy")
pzw_cpla = np.load("./Data/p_cpla_ad.npy")

max_zw_l = 0
min_zw_l = 100000

for idx in range(len(azw)):
    
    error_avg = abs(pzw_avg - azw[idx])
    error_dh = abs(pzw_dh[idx][12] - azw[idx])
    error_cplb = abs(pzw_cplb[12][idx] - azw[idx])
    error_cplu = abs(pzw_cplu[12][idx] - azw[idx])
    error_cpla = abs(pzw_cpla[12][idx] - azw[idx])
    
    t_min = min([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    t_max = max([error_avg, error_dh, error_cplb, error_cplu, error_cpla])
    
    if t_min < min_zw_l:
        
        min_zw_l = t_min
        
    if t_max > max_zw_l:
        
        max_zw_l = t_max
    
print ("ad min loss: ", min_zw_l, "ad max loss: ", max_zw_l)

gd min loss:  [0.00011349] gd max loss:  [339.17166]
ad min loss:  [0.01162195] ad max loss:  [1391.999]


# Mean values for Normalized Mean Absolute Error

## Generated Data

### Fx

In [2]:
afx = np.load("./Data/fx.npy")
afx = afx[int(len(afx)*0.7):]

mean_fx = 0

for idx in range(len(afx)):
    
    mean_fx += abs(afx[idx])
    
mean_fx /= len(afx)

In [3]:
mean_fx

84.70939817976283

## Real Data

### Fx

In [4]:
afx = np.load("./Data/fxr.npy")

mean_fx = 0

for idx in range(len(afx)):
    
    mean_fx += abs(afx[idx])
    
mean_fx /= len(afx)

In [5]:
mean_fx

2282.8645019999262

# Fy

In [6]:
afy = np.load("./Data/fy.npy")
afy = afy[int(len(afy)*0.7):]

mean_fy = 0

for idx in range(len(afy)):
    
    mean_fy += abs(afy[idx])
    
mean_fy /= len(afy)

print ("mean generated fy: ", mean_fy)

afy = np.load("./Data/fyr.npy")

mean_fy = 0

for idx in range(len(afy)):
    
    mean_fy += abs(afy[idx])
    
mean_fy /= len(afy)

print ("mean real fy: ", mean_fy)

mean generated fy:  84.70939817976283
mean real fy:  2281.794340454267


# U0

In [7]:
au0 = np.load("./Data/u0.npy")
au0 = au0[int(len(au0)*0.7):]

mean_u0 = 0

for idx in range(len(au0)):
    
    mean_u0 += abs(au0[idx])
    
mean_u0 /= len(au0)

print ("mean generated u0: ", mean_u0)

au0 = np.load("./Data/u0r.npy")

mean_u0 = 0

for idx in range(len(au0)):
    
    mean_u0 += abs(au0[idx])
    
mean_u0 /= len(au0)

print ("mean real u0: ", mean_u0)

mean generated u0:  56.0
mean real u0:  1042.0412599999809


# V0

In [8]:
av0 = np.load("./Data/v0.npy")
av0 = av0[int(len(av0)*0.7):]

mean_v0 = 0

for idx in range(len(av0)):
    
    mean_v0 += abs(av0[idx])
    
mean_v0 /= len(av0)

print ("mean generated v0: ", mean_v0)

av0 = np.load("./Data/v0r.npy")

mean_v0 = 0

for idx in range(len(av0)):
    
    mean_v0 += abs(av0[idx])
    
mean_v0 /= len(av0)

print ("mean real v0: ", mean_v0)

mean generated v0:  56.0
mean real v0:  529.8880620000258


# Baseline

In [9]:
ab = np.load("./Data/b.npy")
ab = ab[int(len(ab)*0.7):]

mean_b = 0

for idx in range(len(ab)):
    
    mean_b += abs(ab[idx])
    
mean_b /= len(ab)

print ("mean generated b: ", mean_b)

ab = np.load("./Data/br.npy")

mean_b = 0

for idx in range(len(ab)):
    
    mean_b += abs(ab[idx])
    
mean_b /= len(ab)

print ("mean real b: ", mean_b)

mean generated b:  3.1362834089762988
mean real b:  0.20881100000000483


# Disparity

In [10]:
ad = np.load("./Data/d.npy")
ad = ad[int(len(ad)*0.7):]

mean_d = 0

for idx in range(len(ad)):
    
    mean_d += abs(ad[idx])
    
mean_d /= len(ad)

print ("mean generated d: ", mean_d)

ad = np.load("./Data/dr.npy")

mean_d = 0

for idx in range(len(ad)):
    
    mean_d += abs(ad[idx])
    
mean_d /= len(ad)

print ("mean real d: ", mean_d)

mean generated d:  5.6359135359419765
mean real d:  21.47887086050103


# Tx

In [11]:
atx = np.load("./Data/tx.npy")
atx = atx[int(len(atx)*0.7):]

mean_tx = 0

for idx in range(len(atx)):
    
    mean_tx += abs(atx[idx])
    
mean_tx /= len(atx)

print ("mean generated tx: ", mean_tx)

atx = np.load("./Data/txr.npy")

mean_tx = 0

for idx in range(len(atx)):
    
    mean_tx += abs(atx[idx])
    
mean_tx /= len(atx)

print ("mean real tx: ", mean_tx)

mean generated tx:  3.1362834089762988
mean real tx:  2.0


# Ty

In [12]:
aty = np.load("./Data/ty.npy")
aty = aty[int(len(aty)*0.7):]

mean_ty = 0

for idx in range(len(aty)):
    
    mean_ty += abs(aty[idx])
    
mean_ty /= len(aty)

print ("mean generated ty: ", mean_ty)

aty = np.load("./Data/tyr.npy")

mean_ty = 0

for idx in range(len(aty)):
    
    mean_ty += abs(aty[idx])
    
mean_ty /= len(aty)

print ("mean real ty: ", mean_ty)

mean generated ty:  3.2890401748192972
mean real ty:  0.125


# Tz

In [13]:
atz = np.load("./Data/tz.npy")
atz = atz[int(len(atz)*0.7):]

mean_tz = 0

for idx in range(len(atz)):
    
    mean_tz += abs(atz[idx])
    
mean_tz /= len(atz)

print ("mean generated tz: ", mean_tz)

atz = np.load("./Data/tzr.npy")

mean_tz = 0

for idx in range(len(atz)):
    
    mean_tz += abs(atz[idx])
    
mean_tz /= len(atz)

print ("mean real tz: ", mean_tz)

mean generated tz:  2.8612371827197847
mean real tz:  1.2300000000000164


# Pitch

In [14]:
ap = np.load("./Data/p.npy")
ap = ap[int(len(ap)*0.7):]

mean_p = 0

for idx in range(len(ap)):
    
    mean_p += abs(ap[idx])
    
mean_p /= len(ap)

print ("mean generated p: ", mean_p)

ap = np.load("./Data/pr.npy")

mean_p = 0

for idx in range(len(ap)):
    
    mean_p += abs(ap[idx])
    
mean_p /= len(ap)

print ("mean real p: ", mean_p)

mean generated p:  84.54614220877458
mean real p:  0.0219999999999989


# Xw

In [15]:
ax = np.load("./Data/x.npy")
ax = ax[int(len(ax)*0.7):]

mean_x = 0

for idx in range(len(ax)):
    
    mean_x += abs(ax[idx])
    
mean_x /= len(ax)

print ("mean generated x: ", mean_x)

ax = np.load("./Data/xr.npy")

mean_x = 0

for idx in range(len(ax)):
    
    mean_x += abs(ax[idx])
    
mean_x /= len(ax)

print ("mean real x: ", mean_x)

mean generated x:  31.613377053172897
mean real x:  42.289555967764755


# Yw

In [16]:
ay = np.load("./Data/y.npy")
ay = ay[int(len(ay)*0.7):]

mean_y = 0

for idx in range(len(ay)):
    
    mean_y += abs(ay[idx])
    
mean_y /= len(ay)

print ("mean generated y: ", mean_y)

ay = np.load("./Data/yr.npy")

mean_y = 0

for idx in range(len(ay)):
    
    mean_y += abs(ay[idx])
    
mean_y /= len(ay)

print ("mean real y: ", mean_y)

mean generated y:  25.092866140117593
mean real y:  18.34660238283241


# Zw

In [17]:
az = np.load("./Data/z.npy")
az = az[int(len(az)*0.7):]

mean_z = 0

for idx in range(len(az)):
    
    mean_z += abs(az[idx])
    
mean_z /= len(az)

print ("mean generated z: ", mean_z)

az = np.load("./Data/zr.npy")

mean_z = 0

for idx in range(len(az)):
    
    mean_z += abs(az[idx])
    
mean_z /= len(az)

print ("mean real z: ", mean_z)

mean generated z:  42.71013880363322
mean real z:  9.56192501636253


# CVGL Dataset 2.0

In [2]:
data_path = "./Data/"

Left_images = np.load(data_path+"li.npy")
Right_images = np.load(data_path+"ri.npy")
Fx = np.load(data_path+"fx.npy")
Fy = np.load(data_path+"fy.npy") 
U0 = np.load(data_path+"u0.npy") 
V0 = np.load(data_path+"v0.npy") 
Baseline = np.load(data_path+"b.npy")
Disparity = np.load(data_path+"d.npy") 
Tx = np.load(data_path+"tx.npy") 
Ty = np.load(data_path+"ty.npy") 
Tz = np.load(data_path+"tz.npy") 
Pitch = np.load(data_path+"p.npy")
X = np.load(data_path+"x.npy")
Y = np.load(data_path+"y.npy") 
Z = np.load(data_path+"z.npy")


In [3]:
mean_fx = np.mean(Fx[int(len(Fx)*0.7):])

mean_fy = np.mean(Fy[int(len(Fy)*0.7):])

mean_u0 = np.mean(U0[int(len(U0)*0.7):])

mean_v0 = np.mean(V0[int(len(V0)*0.7):])

mean_baseline = np.mean(Baseline[int(len(Baseline)*0.7):])

mean_disparity = np.mean(Disparity[int(len(Disparity)*0.7):])

mean_tx = np.mean(Tx[int(len(Tx)*0.7):])

mean_ty = np.mean(Ty[int(len(Ty)*0.7):])

mean_tz = np.mean(Tz[int(len(Tz)*0.7):])

mean_pitch = np.mean(Pitch[int(len(Pitch)*0.7):])

mean_xw = np.mean(X[int(len(X)*0.7):])

mean_yw = np.mean(Y[int(len(Y)*0.7):])

mean_zw = np.mean(Z[int(len(Z)*0.7):])


In [4]:
mean_fx, mean_fy, mean_u0, mean_v0, mean_baseline, mean_disparity, mean_tx, mean_ty, mean_tz, mean_pitch, mean_xw, mean_yw, mean_zw

(59.32098482132037,
 59.32098482132037,
 56.0,
 56.0,
 -84.00701782907929,
 -10.972388226877689,
 -84.00701782907929,
 0.4372459762640221,
 -0.5766162683574485,
 -12.380371755270145,
 -91.94288567681566,
 0.4372459762640221,
 44.48843272766856)